In [43]:
import numpy as np
import cx_Oracle as ora
import re
import pandas as pd

In [2]:
#eagleproddb.apps.mc.xerox.com
#1523
#EAGLEP
#dsn_tns = cx_Oracle.makedsn('server', 'port', 'sid')
dsn_tns = ora.makedsn('eagleproddb.apps.mc.xerox.com', '1523', 'EAGLEP')
conn = ora.connect(user='siebel_int', password='nov_09', dsn=dsn_tns)
c = conn.cursor()
c.execute("""select sr_num,sr.SR_STAT_ID "Sr Status" ,sr.X_OPERATING_UNIT "Operating Unit", COMM_METHOD_CD "Method", 
sr.recommendation_cd ,sr.REASON_CD "Reason Code", sr.RTNG_CD "Routing Code",ac.created, ac.X_COMMENTS
from siebel.s_srv_req sr, siebel.s_evt_act ac
where sr.row_id = ac.SRA_SR_ID
and ac.TODO_CD = 'Text History'
and sr.comm_method_cd = 'Phone'
and sr.recommendation_cd = 'POC'
--and sr_num in ('1-103156433132')
and sr.created > sysdate - 100
order by SR.sr_num, ac.created, ac.row_id desc""")
#for row in c:
#    print(row)
sr_data = pd.DataFrame(data  = list(c), columns=["SR Number","SR Status","SR_Operating Unit",
                      "SR Method","SR POC/Log","SR Reason Code","SR Routing Code",
                      "TH Created Date","TH_Text History"])
conn.close()

In [3]:
sr_data["Conversation"]= sr_data["TH_Text History"].apply(lambda s :s.split("PROBLEM DETAILS")[1])

In [4]:
len(sr_data)

37589

In [5]:
import os

In [6]:
print(os.getcwd())
os.chdir("C:/Machine Learning/Hackathons/Xerox/Text History Summarization/")
sr_data.to_excel("_Base_Billing_Text_History_100_days.xlsx")

C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\TensorFlow_Framework


In [7]:
sr_data["Conversation"][0]

': . ***** ACCOUNT CLERK REQUESTED MAKE A PAYMENT OF INVOICE # 095255085\n095557006 WITH CREDIT CARD ON FILE THE TOTAL AMOUNT FOR PAYMENT WAS $44.91 PAYMENT SUCCESSFULLY APPROVED. *****'

In [8]:
copy_sr_data = sr_data.copy()

In [107]:
copy_sr_data

,SR Number,SR Status,SR_Operating Unit,SR Method,SR POC/Log,SR Reason Code,SR Routing Code,TH Created Date,TH_Text History,Conversation,Formatted
0,1-101786826463,Closed,Base Inquiry,Phone,POC,Credit Card - Payment Request,None,2019-01-14 14:30:56,** 50035236 - G.GUERRA ** ATLAS UPDATE SR# 1-1...,: . ***** ACCOUNT CLERK REQUESTED MAKE A PAYME...,ACCOUNT CLERK REQUESTED MAKE A PAYMENT OF INV...
1,1-101786826616,Closed,Base Inquiry,Phone,POC,Account Information,None,2019-01-14 14:40:59,** 50035236 - G.GUERRA ** ATLAS UPDATE SR# 1-1...,: . ***** ID#USX30414 CLIENT MANAGER CALLED BE...,ID USX CLIENT MANAGER CALLED BEHALF CUSTOMER ...
2,1-101786826655,Closed,Base Inquiry,Phone,POC,Transfers,None,2019-01-14 15:01:05,** 50035236 - G.GUERRA ** ATLAS UPDATE SR# 1-1...,: . ***** SECRETARY REQUEST ORDER SUPPLIES FOR...,SECRETARY REQUEST ORDER SUPPLIES FOR SERIAL L...
3,1-101786835083,Closed,Base Inquiry,Phone,POC,POC-General,Service Issue,2019-01-14 13:55:27,** 50037341 - J.VILLASENOR ** ATLAS UPDATE SR#...,: . ***** SUSAN CALLED IN TO REPORT THE SHIOP ...,SUSAN CALLED IN TO REPORT THE SHIOP TO ADDRES...
4,1-101786835123,Closed,Base Inquiry,Phone,POC,POC-General,Service Issue,2019-01-14 14:07:41,** 50037341 - J.VILLASENOR ** ATLAS UPDATE SR#...,: . ***** TIM CALLED IN TO HAVE ASSISTANCE WIT...,TIM CALLED IN TO HAVE ASSISTANCE WITH THE ONL...
5,1-101786835465,Closed,Base Inquiry,Phone,POC,POC-General,Form Needed,2019-01-14 14:53:12,** 50037341 - J.VILLASENOR ** ATLAS UPDATE SR#...,: . ***** EDUARDO CALLED IN TO REQUEST TO HAVE...,EDUARDO CALLED IN TO REQUEST TO HAVE THE W FO...
6,1-101786835622,Closed,Base Inquiry,Phone,POC,POC-General,Contract Information,2019-01-14 15:02:54,** 50037341 - J.VILLASENOR ** ATLAS UPDATE SR#...,: . ***** CARMION CALLED IN TO HAVE THE INFORM...,CARMION CALLED IN TO HAVE THE INFORMATION OF ...
7,1-101786835687,Closed,Base Inquiry,Phone,POC,Transfers,None,2019-01-14 15:11:16,** 50037341 - J.VILLASENOR ** ATLAS UPDATE SR#...,: . ***** JAQUELINE CALLED IN TO BE TRANSFERRE...,JAQUELINE CALLED IN TO BE TRANSFERRED TO RBO ...
8,1-101786915813,Closed,Base Inquiry,Phone,POC,POC-General,Contract Information,2019-01-14 13:47:35,** 50021300 - R.GIRON RODAS ** ATLAS UPDATE SR...,: . ***** ANDRE (WC) CALLED IN TO CONFIRM THIS...,ANDRE WC CALLED IN TO CONFIRM THIS MACHINE IS...
9,1-101786915867,Closed,Base Inquiry,Phone,POC,POC-General,Contract Information,2019-01-14 14:01:28,** 50021300 - R.GIRON RODAS ** ATLAS UPDATE SR...,: . ***** DEANNA (XEROX) CALLED IN BEHALF OF T...,DEANNA XEROX CALLED IN BEHALF OF THE CUSTOMER...


In [ ]:
copy_sr_data = copy_sr_data.groupby(["SR Reason Code"]).sum()

In [ ]:
copy_sr_data.reset_index(inplace=True)

In [ ]:
copy_sr_data.loc[0]["Conversation"]

In [108]:
copy_sr_data.groupby(["SR_Operating Unit"]).count()

,SR Number,SR Status,SR Method,SR POC/Log,SR Reason Code,SR Routing Code,TH Created Date,TH_Text History,Conversation,Formatted
SR_Operating Unit,,,,,,,,,,
Base Billing Inquiry,1,1,1,1,1,1,1,1,1,1
Base Inquiry,37584,37584,37584,37584,37584,13980,37584,37584,37584,37584
MS Inquiry,4,4,4,4,4,0,4,4,4,4


In [30]:
print(copy_sr_data.loc[0].values)

['1-101786826463' 'Closed' 'Base Inquiry' 'Phone' 'POC'
 'Credit Card - Payment Request' None Timestamp('2019-01-14 14:30:56')
 '** 50035236 - G.GUERRA ** ATLAS UPDATE SR# 1-101786826463 ** CONTACT IS KERI TEEGARDEN, OTHER, (606) 759-7141 XT6618, INV: 095255085,095557006, SN: MX1212017. RESOLVING CREDIT CARD - PAYMENT REQUEST. PROBLEM DETAILS: . ***** ACCOUNT CLERK REQUESTED MAKE A PAYMENT OF INVOICE # 095255085\n095557006 WITH CREDIT CARD ON FILE THE TOTAL AMOUNT FOR PAYMENT WAS $44.91 PAYMENT SUCCESSFULLY APPROVED. *****'
 ': . ***** ACCOUNT CLERK REQUESTED MAKE A PAYMENT OF INVOICE # 095255085\n095557006 WITH CREDIT CARD ON FILE THE TOTAL AMOUNT FOR PAYMENT WAS $44.91 PAYMENT SUCCESSFULLY APPROVED. *****']


In [41]:
copy_sr_data["Formatted"]= copy_sr_data["Conversation"].apply(lambda s :re.sub('[^a-zA-Z]', ' ',s))
copy_sr_data["Formatted"]= copy_sr_data["Formatted"].apply(lambda s :re.sub(r'\s+', ' ',s))

In [66]:
print(copy_sr_data["Conversation"].loc[9])

: . ***** DEANNA (XEROX) CALLED IN BEHALF OF THE CUSTOMER TO REQUEST INFO ABOUT MACHINE: U88E001228 THIS IS A TEN DIGITS SN SO NO RESULT PROVIDED, BUT CHECKING ON THE BILLING INFO I NOTICE THIS IS A EPSON MACHINE SO I LET THE REP KNOW. SHE STATE SHE HAD A NUMBER TO CALL FOR THOSE SITUATIONS. NO OTHER ACTIONS TAKEN FROM OUR END. *****


In [78]:
from gensim.summarization import summarize
from gensim.summarization import keywords

In [104]:
print(keywords(copy_sr_data["Conversation"].loc[9],ratio = 1))
print (summarize(copy_sr_data["Conversation"].loc[9] , word_count=10))

info
xerox
machine
epson
rep
know
taken
result
provided
actions
billing
request
deanna
called



In [97]:
text = "Thomas A. Anderson is a man living two lives. By day he is an " + \
    "average computer programmer and by night a hacker known as " + \
    "Neo. Neo has always questioned his reality, but the truth is " + \
    "far beyond his imagination. "

print('Input text:')
print (text)

Input text:
Thomas A. Anderson is a man living two lives. By day he is an average computer programmer and by night a hacker known as Neo. Neo has always questioned his reality, but the truth is far beyond his imagination. 


In [105]:
print ('Summary:')
print (summarize(text,word_count=10))

Summary:
Thomas A.
Anderson is a man living two lives.
